In [1]:
import warnings
warnings.filterwarnings('ignore')

In [5]:

import numpy as np
import pandas as pd

from openbb_terminal.sdk import openbb
pd.__version__

'1.5.2'

In [3]:
DATA_STORE=('assets.h5')

In [4]:
sp500_overview = openbb.stocks.screener.screener_data(preset_loaded='sp500_filter', data_type = 'overview')
sp500_ownership = openbb.stocks.screener.screener_data(preset_loaded='sp500_filter', data_type = 'ownership')
sp500_performance = openbb.stocks.screener.screener_data(preset_loaded='sp500_filter', data_type = 'performance')
sp500_technical = openbb.stocks.screener.screener_data(preset_loaded='sp500_filter', data_type = 'technical')
sp500_valuation = openbb.stocks.screener.screener_data(preset_loaded='sp500_filter', data_type = 'valuation')

sp500_overview = sp500_overview.convert_dtypes()
sp500_ownership = sp500_ownership.convert_dtypes()
sp500_performance = sp500_performance.convert_dtypes()
sp500_technical = sp500_technical.convert_dtypes()
sp500_valuation = sp500_valuation.convert_dtypes()

sp500_overview.drop(columns = ['P/E'], inplace = True)
sp500_overview.set_index(keys = ['Ticker', 'Price', 'Change', 'Volume'], inplace = True)
sp500_performance.drop(columns = ['Avg Volume', 'Price', 'Change', 'Volume'], inplace = True)
sp500_performance.set_index(keys = ['Ticker'], inplace = True)
sp500_ownership.drop(columns = ['Price', 'Change', 'Volume', 'Market Cap'], inplace = True)
sp500_ownership.set_index(keys = ['Ticker'], inplace = True)
sp500_technical.drop(columns = ['Price', 'Change', 'Volume'], inplace = True)
sp500_technical.set_index(keys = ['Ticker'], inplace = True)
sp500_valuation.drop(columns = ['Price', 'Change', 'Volume', 'Market Cap'], inplace = True)
sp500_valuation.set_index(keys = ['Ticker'], inplace = True)

sp500_df = sp500_overview.join(sp500_valuation)
sp500_df = sp500_df.join(sp500_ownership)
sp500_df = sp500_df.join(sp500_performance)
sp500_df = sp500_df.join(sp500_technical)

sp500_df.reset_index(inplace = True)

sp500_df

Error: 'sp500_filter'

Error: 'sp500_filter'

Error: 'sp500_filter'

Error: 'sp500_filter'

Error: 'sp500_filter'

AttributeError: 'list' object has no attribute 'convert_dtypes'